In [1]:
# Library
import numpy as np
import cv2
import os
import glob

# Otsu Method
from numpy import asarray

# GLCM
from skimage.feature import greycomatrix, greycoprops

In [2]:
def compute_otsu_criteria(im, th):
    # create the thresholded image
    thresholded_im = np.zeros(im.shape)
    thresholded_im[im >= th] = 1

    # compute weights
    nb_pixels = im.size
    nb_pixels1 = np.count_nonzero(thresholded_im)
    weight1 = nb_pixels1 / nb_pixels
    weight0 = 1 - weight1

    # if one the classes is empty, eg all pixels are below or above the threshold, that threshold will not be considered
    # in the search for the best threshold
    if weight1 == 0 or weight0 == 0:
        return np.inf

    # find all pixels belonging to each class
    val_pixels1 = im[thresholded_im == 1]
    val_pixels0 = im[thresholded_im == 0]

    # compute variance of these classes
    var0 = np.var(val_pixels0) if len(val_pixels0) > 0 else 0
    var1 = np.var(val_pixels1) if len(val_pixels1) > 0 else 0

    return weight0 * var0 + weight1 * var1

def preprocess(path_src, path_th, path_dl, path_er, path_gray, label):
    i = 0
    for image in path_src:
        img = cv2.imread(image)
        im = asarray(img)

        # testing all thresholds from 0 to the maximum of the image
        threshold_range = range(np.max(im)+1)
        criterias = [compute_otsu_criteria(im, th) for th in threshold_range]

        # best threshold is the one minimizing the Otsu criteria
        best_threshold = threshold_range[np.argmin(criterias)]
        print(best_threshold)

        grayscale = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ret, img1 = cv2.threshold(grayscale, best_threshold, 255, cv2.THRESH_BINARY_INV)
        cv2.imwrite(path_th + "image%02i.jpg" %i, img1)
        
        # Closing Operation
        # Dilation
        img1 = cv2.dilate(img1.copy(), None, iterations=5)
        cv2.imwrite(path_dl + "image%02i.jpg" %i, img1)
        
        # Erosion
        img1 = cv2.erode(img1.copy(), None, iterations=5)
        cv2.imwrite(path_er + "image%02i.jpg" %i, img1)
        
        
        b, g, r = cv2.split(img)
        rgba = [b,g,r,img1]
        dst = cv2.merge(rgba, 4)
        
        contours, hierarchy = cv2.findContours(img1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        select = max(contours, key=cv2.contourArea)
        x,y,w,h = cv2.boundingRect(select)
        png = dst[y:y+h,x:x+w]
        gray = cv2.cvtColor(png, cv2.COLOR_BGR2GRAY)
        grays.append(gray)
        labels.append(label)
        cv2.imwrite(path_gray + "image%02i.jpg" %i, gray)    
        i+=1
        print ("Berhasil")
    
    print("Berhasil save")
    return grays

In [3]:
grays = [] # Container Image
labels = [] # Container Label (Positive or Negative)

### Import Image and Save the Result

Status PJK Negatif

In [4]:
path_src = glob.glob("Image/Negatif/*.jpg")
path_th = "Image/Thresholding/Negatif/"
path_dl = "Image/Dilasi/Negatif/"
path_er ="Image/Erosi/Negatif/"
path_gray ="Image/Gray/Negatif/"
label = "Negatif"

preprocess(path_src, path_th, path_dl, path_er, path_gray, label)

110
Berhasil
152
Berhasil
143
Berhasil
Berhasil save


[array([[ 73,  73,  73, ...,  52,  52,  52],
        [ 69,  69,  69, ...,  53,  53,  52],
        [ 68,  68,  67, ...,  54,  53,  53],
        ...,
        [153, 163, 168, ...,  94,  94,  97],
        [165, 170, 167, ...,  90,  92,  95],
        [164, 168, 157, ...,  90,  93,  98]], dtype=uint8),
 array([[115, 115, 115, ..., 187, 188, 190],
        [115, 116, 117, ..., 183, 184, 187],
        [112, 113, 115, ..., 183, 184, 186],
        ...,
        [127, 127, 127, ..., 192, 192, 187],
        [128, 127, 126, ..., 191, 190, 187],
        [129, 127, 126, ..., 189, 188, 189]], dtype=uint8),
 array([[119, 117, 114, ..., 142, 140, 143],
        [116, 115, 114, ..., 146, 144, 146],
        [113, 114, 114, ..., 153, 150, 150],
        ...,
        [158, 163, 161, ..., 159, 155, 157],
        [146, 154, 157, ..., 154, 157, 159],
        [141, 145, 153, ..., 156, 159, 161]], dtype=uint8)]

Status PJK Positif

In [5]:
path_src = glob.glob("Image/Postif/*.jpg")
path_th = "Image/Thresholding/Positif/"
path_dl = "Image/Dilasi/Positif/"
path_er ="Image/Erosi/Positif/"
path_gray ="Image/Gray/Positif/"
label = "Positif"

preprocess(path_src, path_th, path_dl, path_er, path_gray, label)

Berhasil save


[array([[ 73,  73,  73, ...,  52,  52,  52],
        [ 69,  69,  69, ...,  53,  53,  52],
        [ 68,  68,  67, ...,  54,  53,  53],
        ...,
        [153, 163, 168, ...,  94,  94,  97],
        [165, 170, 167, ...,  90,  92,  95],
        [164, 168, 157, ...,  90,  93,  98]], dtype=uint8),
 array([[115, 115, 115, ..., 187, 188, 190],
        [115, 116, 117, ..., 183, 184, 187],
        [112, 113, 115, ..., 183, 184, 186],
        ...,
        [127, 127, 127, ..., 192, 192, 187],
        [128, 127, 126, ..., 191, 190, 187],
        [129, 127, 126, ..., 189, 188, 189]], dtype=uint8),
 array([[119, 117, 114, ..., 142, 140, 143],
        [116, 115, 114, ..., 146, 144, 146],
        [113, 114, 114, ..., 153, 150, 150],
        ...,
        [158, 163, 161, ..., 159, 155, 157],
        [146, 154, 157, ..., 154, 157, 159],
        [141, 145, 153, ..., 156, 159, 161]], dtype=uint8)]

In [6]:
print(len(grays))

3


In [7]:
# cv2.imshow("gray image",grays[5])
# cv2.waitKey(0) 
  
# #closing all open windows 
# cv2.destroyAllWindows() 

In [8]:
def calc_glcm_all_agls(gray, label, props, dists=[5], agls=[0, np.pi/4, np.pi/2, 3*np.pi/4], lvl=256, sym=True, norm=True):
    
    glcm = greycomatrix(img, 
                        distances=dists, 
                        angles=agls, 
                        levels=lvl,
                        symmetric=sym, 
                        normed=norm)
    feature = []
    glcm_props = [propery for name in props for propery in greycoprops(glcm, name)[0]]
    for item in glcm_props:
            feature.append(item)
    feature.append(label) 
    
    return feature

properties = ['dissimilarity', 'correlation', 'homogeneity', 'contrast', 'ASM', 'energy']
columns = []
angles = ['0', '45', '90','135']
for name in properties :
    for ang in angles:
        columns.append(name + "_" + ang)
        
columns.append("label")


glcm_all_agls = []
for img, label in zip(grays, labels): 
    glcm_all_agls.append(
            calc_glcm_all_agls(img, 
                                label, 
                                props=properties)
                            )
    


In [9]:
import pandas as pd 

# Create the pandas DataFrame for GLCM features data
glcm_df = pd.DataFrame(glcm_all_agls,
                      columns = columns)

glcm_df.to_csv("dataset_feature.csv", sep=';', encoding='utf-8', index=False)
glcm_df

,dissimilarity_0,dissimilarity_45,dissimilarity_90,dissimilarity_135,correlation_0,correlation_45,correlation_90,correlation_135,homogeneity_0,homogeneity_45,...,contrast_135,ASM_0,ASM_45,ASM_90,ASM_135,energy_0,energy_45,energy_90,energy_135,label
0,7.034388,7.640496,7.663753,7.445453,0.923183,0.909970,0.909637,0.915908,0.178868,0.171162,...,121.783663,0.000452,0.000427,0.000428,0.000422,0.021252,0.020661,0.020698,0.020554,Negatif
1,5.020745,5.339807,5.145619,5.236546,0.930061,0.925388,0.929417,0.923329,0.220281,0.204911,...,61.679466,0.000836,0.000779,0.000808,0.000807,0.028916,0.027908,0.028425,0.028406,Negatif
2,6.268082,6.986674,6.975382,7.089005,0.945891,0.935333,0.936414,0.933253,0.173524,0.158174,...,95.162514,0.000555,0.000501,0.000505,0.000500,0.023551,0.022394,0.022481,0.022369,Negatif


SyntaxError: invalid syntax (<ipython-input-12-80ba375e8324>, line 1)